# Fine-Tune "sentence-transformers/paraphrase-MiniLM-L3-v2" Sentence Transformer

In [152]:
modelName= "sentence-transformers/paraphrase-MiniLM-L3-v2"

In [153]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn, optim
from sentence_transformers import SentenceTransformer, models, losses, InputExample, models
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from transformers import AutoTokenizer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [154]:
df = pd.read_csv("datasets/resume.csv")  # Replace with actual path
df = df[["Resume_str", "Category"]]

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])

texts = df['Resume_str'].tolist()
labels = df['label'].tolist()

In [155]:
class ResumeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.tokenized = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized["input_ids"][idx],
            "attention_mask": self.tokenized["attention_mask"][idx],
            "label": self.labels[idx]
        }

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
dataset = ResumeDataset(texts, labels, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [156]:
# Step 1: Build SentenceTransformer-style base model
word_embedding_model = models.Transformer(modelName)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
base_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Step 2: Define classification head
class SentenceClassifier(nn.Module):
    def __init__(self, base_model, num_classes):
        super(SentenceClassifier, self).__init__()
        self.base_model = base_model  # SentenceTransformer model
        self.classifier = nn.Sequential(
            nn.Linear(base_model.get_sentence_embedding_dimension(), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, features):
        # Allow gradients for fine-tuning base model
        embeddings = self.base_model.forward(features)['sentence_embedding']
        return self.classifier(embeddings)

# Step 3: Instantiate classifier model
model = SentenceClassifier(base_model, num_classes=len(label_encoder.classes_))

In [157]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Early Stopping Setup
best_loss = float('inf')
patience = 6
counter = 0

# Training config
baseEpochs = 200
device = torch.device('cuda' if torch.cuda.is_available else 'mps' if torch.mps.is_available else 'cpu')
print("Using device:", device)
model = model.to(device)

# Freeze base model initially
for param in model.base_model.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4)

# Learning Rate Scheduler (optional for frozen phase)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

torch.save(model.state_dict(), "models/best_model.pt")
for epoch in range(baseEpochs):
    model.load_state_dict(torch.load("models/best_model.pt"))
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = torch.tensor(batch["label"]).to(device)

        features = {"input_ids": input_ids, "attention_mask": attention_mask}
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} complete. Avg Loss: {avg_loss:.4f}")

    # Step the scheduler
    scheduler.step(avg_loss)

    # Early Stopping Check
    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        print("✅ Improvement detected — saving model")
        torch.save(model.state_dict(), "models/best_model.pt")
    else:
        counter += 1
        print(f"⚠️ No improvement. Patience left: {patience - counter}")
        if counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

Using device: cuda


Epoch 1:   0%|          | 0/156 [00:00<?, ?it/s]<ipython-input-157-f8a4cc2551eb>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"]).to(device)


Epoch 1 complete. Avg Loss: 3.1331
✅ Improvement detected — saving model


Epoch 2 complete. Avg Loss: 3.0254
✅ Improvement detected — saving model


Epoch 3 complete. Avg Loss: 2.8772
✅ Improvement detected — saving model


...

Epoch 87 complete. Avg Loss: 1.0872
⚠️ No improvement. Patience left: 4


Epoch 88 complete. Avg Loss: 1.0792
⚠️ No improvement. Patience left: 3


Epoch 89 complete. Avg Loss: 1.0928
⚠️ No improvement. Patience left: 2


Epoch 90 complete. Avg Loss: 1.0853
⚠️ No improvement. Patience left: 1


Epoch 91 complete. Avg Loss: 1.0804
⚠️ No improvement. Patience left: 0
⏹️ Early stopping triggered.


In [158]:
tuneEpochs = 20

# 🔓 Unfreeze base model
for param in model.base_model.parameters():
    param.requires_grad = True

# 🔁 New optimizer & scheduler for fine-tuning
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Early stopping state
tune_best_loss = float('inf')
tune_counter = 0
tune_patience = 6

for epoch in range(baseEpochs, baseEpochs + tuneEpochs):
    model.load_state_dict(torch.load("models/best_model.pt"))
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Tune Epoch {epoch+1}", leave=False)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = torch.tensor(batch["label"]).to(device)

        features = {"input_ids": input_ids, "attention_mask": attention_mask}
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"🛠️ Epoch {epoch+1} complete. Avg Loss: {avg_loss:.4f}")

    scheduler.step(avg_loss)

    # Early stopping
    if avg_loss < tune_best_loss:
        tune_best_loss = avg_loss
        tune_counter = 0
        print("✅ Improvement detected — saving model")
        torch.save(model.state_dict(), "models/best_model.pt")
    else:
        tune_counter += 1
        print(f"⚠️ No improvement. Patience left: {tune_patience - tune_counter}")
        if tune_counter >= tune_patience:
            print("⏹️ Early stopping triggered.")
            break

Tune Epoch 201:   0%|          | 0/156 [00:00<?, ?it/s]<ipython-input-158-04f3c0236ff8>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"]).to(device)


🛠️ Epoch 201 complete. Avg Loss: 1.0300
✅ Improvement detected — saving model


🛠️ Epoch 202 complete. Avg Loss: 0.7516
✅ Improvement detected — saving model


🛠️ Epoch 203 complete. Avg Loss: 0.6303
✅ Improvement detected — saving model


🛠️ Epoch 204 complete. Avg Loss: 0.5337
✅ Improvement detected — saving model


...

🛠️ Epoch 216 complete. Avg Loss: 0.1046
⚠️ No improvement. Patience left: 5


🛠️ Epoch 217 complete. Avg Loss: 0.1034
⚠️ No improvement. Patience left: 4


🛠️ Epoch 218 complete. Avg Loss: 0.1032
⚠️ No improvement. Patience left: 3


🛠️ Epoch 219 complete. Avg Loss: 0.1047
⚠️ No improvement. Patience left: 2


🛠️ Epoch 220 complete. Avg Loss: 0.0799
✅ Improvement detected — saving model


In [162]:
model.load_state_dict(torch.load("models/best_model.pt"))
torch.save(model.base_model.state_dict(), "models/model2.pt")

# TEST

In [163]:
encoder = SentenceTransformer(modelName)
resume = "Master's in Computer Science"
jd = "Bachelor's in Healthcare or related fields"
resumeEmbeddings = encoder.encode([resume])
jdEmbeddings = encoder.encode([jd])
similarity = cosine_similarity(resumeEmbeddings, jdEmbeddings)[0][0]
print("Similarity : ", similarity)

Similarity :  0.2797977


In [164]:
encoder = SentenceTransformer(modelName)
encoder.load_state_dict(torch.load("models/model2.pt"))
resume = "Master's in Computer Science"
jd = "Bachelor's in Healthcare or related fields"
resumeEmbeddings = encoder.encode([resume])
jdEmbeddings = encoder.encode([jd])
similarity = cosine_similarity(resumeEmbeddings, jdEmbeddings)[0][0]
print("Similarity : ", similarity)

Similarity :  0.3099702
